In [ ]:
# EDA_Chinook.ipynb
# -*- coding: utf-8 -*-

# === IMPORTS ===
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# === CONFIG DATABASE ===
user = "root"
password = "root"
host = "localhost"
port = 3306
database = "Chinook"

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}")

# === LOAD TABLES ===
tables = ["Artist", "Album", "Track", "Genre", "Customer", "Invoice", "InvoiceLine", "Playlist", "PlaylistTrack", "Employee", "MediaType"]

data = {}
for table in tables:
    data[table] = pd.read_sql(f"SELECT * FROM {table}", engine)
    print(f"{table} loaded, shape: {data[table].shape}")

# === GENERAL INFO ===
for table, df in data.items():
    print(f"\n=== {table} ===")
    display(df.head())
    print(df.info())
    print(df.describe(include='all'))

# === EXPLORATORY VISUALIZATION ===

# 1. Top 10 genres by number of tracks
plt.figure(figsize=(12,6))
genre_count = data['Track'].groupby('GenreId').size().sort_values(ascending=False).head(10)
genre_names = data['Genre'].set_index('GenreId').loc[genre_count.index]['Name']
sns.barplot(x=genre_names, y=genre_count.values, palette="viridis")
plt.title("Top 10 Genres by Number of Tracks")
plt.ylabel("Number of Tracks")
plt.xlabel("Genre")
plt.xticks(rotation=45)
plt.show()

# 2. Top 10 artists by number of albums
plt.figure(figsize=(12,6))
artist_album_count = data['Album'].groupby('ArtistId').size().sort_values(ascending=False).head(10)
artist_names = data['Artist'].set_index('ArtistId').loc[artist_album_count.index]['Name']
sns.barplot(x=artist_names, y=artist_album_count.values, palette="coolwarm")
plt.title("Top 10 Artists by Number of Albums")
plt.ylabel("Number of Albums")
plt.xlabel("Artist")
plt.xticks(rotation=45)
plt.show()

# 3. Total sales by country
invoice_country = data['Invoice'].groupby('BillingCountry')['Total'].sum().sort_values(ascending=False)
plt.figure(figsize=(12,6))
sns.barplot(x=invoice_country.index, y=invoice_country.values, palette="magma")
plt.title("Total Sales by Country")
plt.ylabel("Total Sales")
plt.xlabel("Country")
plt.xticks(rotation=45)
plt.show()

# 4. Distribution of track prices
plt.figure(figsize=(8,5))
sns.histplot(data['Track']['UnitPrice'], bins=20, kde=True, color='skyblue')
plt.title("Distribution of Track Prices")
plt.xlabel("Price ($)")
plt.ylabel("Count")
plt.show()

# 5. Distribution of track durations (in minutes)
plt.figure(figsize=(8,5))
track_minutes = data['Track']['Milliseconds'] / 60000
sns.histplot(track_minutes, bins=20, kde=True, color='salmon')
plt.title("Distribution of Track Durations (minutes)")
plt.xlabel("Duration (minutes)")
plt.ylabel("Count")
plt.show()
